In [1]:
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import Normalizer 
import numpy as np
import matplotlib.pyplot as plt

C:\Users\verse\AppData\Local\Temp\ipykernel_35240\3609143969.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [89]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [90]:
%matplotlib inline

In [118]:
with open("Video_Games.csv", "r") as in_file:
    df = pd.read_csv(in_file)

In [128]:
df = df.fillna(0)
df = df.replace('tbd', 0)

In [120]:
df.head(4)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.0,0.0,0,0.0,0,0
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E


In [121]:
df = df.drop(['Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Critic_Count', 'User_Count', 'Developer', 'Rating'], axis=1)

In [129]:
import pandas as pd

df['User_Score'] = df['User_Score'].astype(float)

# Get unique genres
unique_genres = df['Genre'].unique()

# Add binary columns for each unique genre
for genre in unique_genres:
    df[genre] = (df['Genre'] == genre).astype(int)

df.head()




,Name,Platform,Year_of_Release,Genre,Global_Sales,Critic_Score,User_Score,Sports,Racing,Role-Playing,Puzzle,Misc,Shooter,Simulation,Action,Fighting,Adventure,Strategy,0
0,Wii Sports,0,2006.0,Sports,82.53,76.0,8.0,1,0,0,0,0,0,0,0,0,0,0,0
1,Super Mario Bros.,1,1985.0,Platform,40.24,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,Mario Kart Wii,0,2008.0,Racing,35.52,82.0,8.3,0,1,0,0,0,0,0,0,0,0,0,0
3,Wii Sports Resort,0,2009.0,Sports,32.77,80.0,8.0,1,0,0,0,0,0,0,0,0,0,0,0
4,Pokemon Red/Pokemon Blue,0,1996.0,Role-Playing,31.37,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0


In [134]:
df_norm = pd.DataFrame(df)

# Selecting only the numeric columns for normalization
numeric_cols = df.select_dtypes(include=['float64', 'int64', 'int']).columns

print(numeric_cols)

# Min-max scaling
df[numeric_cols] = (df[numeric_cols] - df[numeric_cols].min()) / (df[numeric_cols].max() - df[numeric_cols].min())

df

Index([       'Platform', 'Year_of_Release',    'Global_Sales',
          'Critic_Score',      'User_Score',          'Sports',
                'Racing',    'Role-Playing',          'Puzzle',
                  'Misc',         'Shooter',      'Simulation',
                'Action',        'Fighting',       'Adventure',
              'Strategy',                 0],
      dtype='object')


,Name,Platform,Year_of_Release,Genre,Global_Sales,Critic_Score,User_Score,Sports,Racing,Role-Playing,Puzzle,Misc,Shooter,Simulation,Action,Fighting,Adventure,Strategy,0
0,Wii Sports,0.0,0.993069,Sports,1.000000,0.775510,0.824742,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Super Mario Bros.,1.0,0.982673,Platform,0.487518,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Mario Kart Wii,0.0,0.994059,Racing,0.430320,0.836735,0.855670,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Wii Sports Resort,0.0,0.994554,Sports,0.396995,0.816327,0.824742,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Pokemon Red/Pokemon Blue,0.0,0.988119,Role-Playing,0.380029,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,0.0,0.998020,Action,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
16715,LMA Manager 2007,0.0,0.993069,Sports,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16716,Haitaka no Psychedelica,0.0,0.998020,Adventure,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
16717,Spirits & Spells,1.0,0.991584,Platform,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
from scipy.spatial.distance import euclidean

# Target game
target_game = df[df['Name'] == 'Wii Sports'].iloc[0]

# Compute Euclidean distances
distances = {}
for index, row in df.iterrows():
    if row['Name'] != 'Wii Sports':
        distances[row['Name']] = euclidean(target_game[numeric_cols], row[numeric_cols])

# Sort distances and select 10 most similar games
similar_games = sorted(distances.items(), key=lambda x: x[1])[:10]

# Display the result
print("10 most similar games to 'Wii Sports':")
for game, distance in similar_games:
    print(f"- {game} (Distance: {distance})")

10 most similar games to 'Wii Sports':
- Wii Sports Resort (Distance: 0.604386969289627)
- Wii Fit (Distance: 0.7268428058263842)
- Wii Fit Plus (Distance: 0.7397867877724733)
- Kinect Sports (Distance: 0.9276820230062173)
- Sports Champions (Distance: 0.953644790547385)
- EA Sports Active (Distance: 0.9569509291617333)
- Mario Strikers Charged (Distance: 0.969339657176511)
- Hot Shots Golf 3 (Distance: 0.9738920295405831)
- Skate 3 (Distance: 0.9778518867395347)
- Mario Super Sluggers (Distance: 0.9800420757004247)


In [132]:
from scipy.spatial.distance import euclidean

# Target game
target_game = df[df['Name'] == 'FIFA 14'].iloc[0]

# Compute Euclidean distances
distances = {}
for index, row in df.iterrows():
    if row['Name'] != 'FIFA 14':
        distances[row['Name']] = euclidean(target_game[numeric_cols], row[numeric_cols])

# Sort distances and select 10 most similar games
similar_games = sorted(distances.items(), key=lambda x: x[1])[:10]

# Display the result
print("10 most similar games to 'FIFA 14':")
for game, distance in similar_games:
    print(f"- {game} (Distance: {distance})")

10 most similar games to 'FIFA 14':
- FIFA 17 (Distance: 0.08028974346314427)
- FIFA 15 (Distance: 0.08601928094084807)
- FIFA 16 (Distance: 0.09210615668067239)
- Tiger Woods PGA Tour 2005 (Distance: 0.09579379299863768)
- Tiger Woods PGA Tour 06 (Distance: 0.10138449390478362)
- NHL 17 (Distance: 0.12285152311956261)
- College Hoops 2K8 (Distance: 0.12715264481463667)
- Pro Evolution Soccer 2016 (Distance: 0.12839358806023898)
- 2014 FIFA World Cup Brazil (Distance: 0.1430924533838698)
- Madden NFL 13 (Distance: 0.14956685180815002)


In [133]:
from scipy.spatial.distance import euclidean

# Target game
target_game = df[df['Name'] == 'Left 4 Dead'].iloc[0]

# Compute Euclidean distances
distances = {}
for index, row in df.iterrows():
    if row['Name'] != 'Left 4 Dead':
        distances[row['Name']] = euclidean(target_game[numeric_cols], row[numeric_cols])

# Sort distances and select 10 most similar games
similar_games = sorted(distances.items(), key=lambda x: x[1])[:10]

# Display the result
print("10 most similar games to 'Left 4 Dead':")
for game, distance in similar_games:
    print(f"- {game} (Distance: {distance})")

10 most similar games to 'Left 4 Dead':
- Red Faction (Distance: 0.024365367214869104)
- Killzone 2 (Distance: 0.029640786522318878)
- Resistance: Fall of Man (Distance: 0.0382672373185901)
- Gears of War 3 (Distance: 0.0399441519403831)
- Crysis (Distance: 0.04091874384084491)
- Battlefield 3 (Distance: 0.04220145642291829)
- Far Cry 3 (Distance: 0.04493867271885976)
- Far Cry (Distance: 0.046766508662262264)
- Uncharted 4: A Thief's End (Distance: 0.046852812212276954)
- Gears of War (Distance: 0.04808012532602405)
